In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("tree_example").getOrCreate()

24/01/12 22:09:12 WARN Utils: Your hostname, DESKTOP-0HJMRJF resolves to a loopback address: 127.0.1.1; using 172.25.126.180 instead (on interface eth0)
24/01/12 22:09:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/12 22:09:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/12 22:09:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/01/12 22:09:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

In [4]:
data = spark.read.format("libsvm").load('./data/ML/logistic/sample_libsvm_data.txt')

24/01/12 22:12:57 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+


In [6]:
trainData, testData = data.randomSplit([0.8, 0.2])

In [7]:
dtc = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [24]:
rfc = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=20 )

In [25]:
gbt = GBTClassifier(labelCol="label", featuresCol="features")

In [26]:
dtc_model = dtc.fit(trainData)
rfc_model = rfc.fit(trainData)
gbt_model = gbt.fit(trainData)

In [29]:
dtc_preds = dtc_model.transform(testData)
rfc_preds = rfc_model.transform(testData)
gbt_preds = gbt_model.transform(testData)

In [30]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[126,127,128...|   [36.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[128,129,130...|   [36.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[129,130,131...|   [36.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[150,151,152...|   [36.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [36.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[154,155,156...|   [36.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[154,155,156...|   [0.0,46.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[100,101,102...|   [0.0,46.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[124,125,126...|   [0.0,46.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[125,126,153...|   [0.0,46.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[127,128,129...|   [0.0,46.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[127,128,155...|   [0.0,46.0]|  [0.0,1.0]|       1.0|
|  1.0|(69

In [31]:
rfc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[126,127,128...|   [20.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[128,129,130...|   [20.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[129,130,131...|   [17.0,3.0]|[0.85,0.15]|       0.0|
|  0.0|(692,[150,151,152...|   [18.0,2.0]|  [0.9,0.1]|       0.0|
|  0.0|(692,[152,153,154...|   [18.0,2.0]|  [0.9,0.1]|       0.0|
|  0.0|(692,[154,155,156...|   [18.0,2.0]|  [0.9,0.1]|       0.0|
|  0.0|(692,[154,155,156...|   [16.0,4.0]|  [0.8,0.2]|       0.0|
|  1.0|(692,[100,101,102...|   [0.0,20.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[124,125,126...|   [0.0,20.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[125,126,153...|   [0.0,20.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[127,128,129...|   [0.0,20.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[127,128,155...|   [0.0,20.0]|  [0.0,1.0]|       1.0|
|  1.0|(69

In [32]:
gbt_preds.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[126,127,128...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[128,129,130...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[129,130,131...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[150,151,152...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[152,153,154...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[154,155,156...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[154,155,156...|[-1.2142407849861...|[0.08102645939004...|       1.0|
|  1.0|(692,[100,101,102...|[-1.5435020027249...|[0.04364652142729...|       1.0|
|  1.0|(692,[124,125,126...|[-1.5435020027249...|[0.04364652142729...|       1.0|
|  1.0|(692,[125

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
acc_eval = MulticlassClassificationEvaluator(metricName="accuracy")

In [35]:
acc_eval.evaluate(dtc_preds)

0.9411764705882353

In [36]:
acc_eval.evaluate(rfc_preds)


1.0

In [37]:
acc_eval.evaluate(gbt_preds)

0.9411764705882353

In [39]:
# rfc_model.featureImportances